In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('data.csv', encoding='cp1251', delimiter = ";", index_col="№ страхового события")
data = data[["Участник 1", "Участник 2"]]
data.head()

,Участник 1,Участник 2
№ страхового события,,
1,Беляев Владислав Аркадьевич,Ефимов Малик Константинович
2,Давыдова Ануш Оскаровна,Никифорова Божена Львовна
3,Белов Сергей Михайлович,Третьяков Никита Харитонович
4,Власов Артём Михайлович,Аксенова Вероника Кузьминична
5,Яковлев Алан Макарович,Сысоев Елисей Тимофеевич


Собственно говоря у нас имеются только имена участников. Как понять, кто из них мошенник - не знаю. Единственная моя догадка, это то, что мошенники часто попадают в ДТП

In [3]:
data.shape

(261, 2)

In [4]:
data['Участник 1'].nunique()

256

In [5]:
data['Участник 2'].nunique()

258

In [6]:
data['Участник 1'].value_counts()[:4]

Дорофеева Гулия Николаевна         3
Стрелков Евгений Егорович          2
Мухамадеев Александр Валерьевич    2
Котова Виктория Архиповна          2
Name: Участник 1, dtype: int64

In [7]:
data['Участник 2'].value_counts()[:3]

Сенчукова Екатерина Семеновна    2
Дорофеева Гулия Николаевна       2
Мамонтов Мстислав Георгиевич     2
Name: Участник 2, dtype: int64

Дорофеева точно ведёт себя крайне странно. Возможно ещё следует в целом посмотреть на пересечения двух массивов

In [8]:
all_people = pd.concat([data['Участник 1'], data['Участник 2']])

In [9]:
all_people.value_counts()[:11]

Дорофеева Гулия Николаевна         5
Котова Виктория Архиповна          3
Мамонтов Мстислав Георгиевич       3
Павлова Мария Геннадиевна          2
Мухамадеев Александр Валерьевич    2
Стрелков Евгений Егорович          2
Рогачев Антон Владимирович         2
Сенчукова Екатерина Семеновна      2
Комин Сергей Николаевич            2
Коробов Вадим Александрович        2
Воробьев Иван Александрович        2
dtype: int64

Котова и Мамонтов тоже могут быть жуликами. И последнее предположение которое у меня есть: друзья жуликов - жулики

In [10]:
cheater = set(["Дорофеева Гулия Николаевна", "Котова Виктория Архиповна", "Мамонтов Мстислав Георгиевич"])

In [11]:
num = 0
while len(cheater) > num:
    num = len(cheater)
    for person in data.values:
        if person[0] in cheater:
            cheater.add(person[1])
        if person[1] in cheater:
            cheater.add(person[0])
print(cheater)

{'Котова Виктория Архиповна', 'Мамонтов Мстислав Георгиевич', 'Суворов Илья Макарович', 'Королева Радмила Олеговна', 'Цветков Валерий Ростиславович', 'Архипова Анна Игоревна', 'Галкина Елена Тарасовна', 'Петухова Медина Максимовна', 'Шубин Всеволод Валерьевич', 'Федотов Ростислав Богуславович', 'Дорофеева Гулия Николаевна', 'Миронов Пётр Александрович', 'Елисеева Сати Вячеславовна', 'Пахомов Анатолий Васильевич'}


В итоге можно сказать, что у нас есть 3 почти 100% мошенников  и всего 14 людей, которые могли жульничать. Какой из этого всего можно сделать вывод. Нам важны люди, у которых большое число связей и все их связи(а ещё связи связи связей и т.д.) По сути мы занимаемся кластеризацией.
А говоря математическим языком мы ищем компоненты связности в графе. К сожалению я не пользовался алгоритмами связанными с этим ранее.
Если бы выборка была большой, то наверное пришлось бы строить бы строить матрицу смежности. И работать с ней.
<a href="https://habrahabr.ru/post/112421/">Статья про графы в питоне</a>